In [1]:
import xml.etree.ElementTree as ET
import csv
import pandas as pd
import traceback
import sys
import numpy as np


In [2]:
#common variables

CUTTOFF=100000000
LOGAMOUNT=1000000

OUTPUT1="output/output_1.csv"
OUTPUT2="output/output_2.csv"
OUTPUT3="output/output_3.csv"
OUTPUT4="output/output_4.csv"
PROCESSEDFINAL="output/processed_body.csv"
EXISTING_TAGS="output/existing_tags.csv"
TAGSFILE = 'datadumps/Tags.xml'
POSTSFILE = 'datadumps/Posts.xml'

In [3]:
tags_context = ET.iterparse(TAGSFILE, events=("start", "end"),
                       parser=ET.XMLParser(encoding='utf-8'))

In [4]:
# lets look at the tags
_, root = next(tags_context)
for index, item in enumerate(tags_context):
    event, elem = item
    if event == "end" and elem.tag == "row":
        print(elem.attrib)
        print('\n')
        
        if index >1 and index!= 0:
            break
    
    

{'Id': '1', 'TagName': '.net', 'Count': '310793', 'ExcerptPostId': '3624959', 'WikiPostId': '3607476'}


{'Id': '2', 'TagName': 'html', 'Count': '1088688', 'ExcerptPostId': '3673183', 'WikiPostId': '3673182'}




In [5]:
with open(EXISTING_TAGS, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['TAG'])
    _, root = next(tags_context)
    for index, item in enumerate(tags_context):
        event, elem = item
        if event == "end" and elem.tag == "row":
            tag = elem.attrib.get('TagName', 'None')
            csvwriter.writerow([tag])

In [6]:
search_for_tags_df =pd.read_csv(EXISTING_TAGS)
search_for_tags_df = search_for_tags_df.sort_values('TAG')
search_for_tags_df

,TAG
16566,.a
15157,.app
16357,.aspxauth
11751,.bash-profile
15329,.class-file
...,...
36586,zynq
55385,zynq-ultrascale+
29917,zypper
352,NaN


In [7]:
context = ET.iterparse(POSTSFILE, events=("start", "end"),
                       parser=ET.XMLParser(encoding='utf-8'))

In [8]:
# lets look at the data
_, root = next(context)
for index, item in enumerate(context):
    event, elem = item
    if event == "end" and elem.tag == "row":
        print(elem.attrib)
        print('\n')
        tags = elem.attrib.get('Tags', 'None')
        #remove all '<', '>'
        reworked_tags = tags[1:len(tags)-1].split('><')
        print('tags = ', reworked_tags)
        print('\n')
        
    if index >1 and index!= 0:
        break
    
    


{'Id': '4', 'PostTypeId': '1', 'AcceptedAnswerId': '7', 'CreationDate': '2008-07-31T21:42:52.667', 'Score': '735', 'ViewCount': '59537', 'Body': '<p>I want to use a <code>Track-Bar</code> to change a <code>Form</code>\'s opacity.</p>\n<p>This is my code:</p>\n<pre class="lang-cs prettyprint-override"><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n<p>When I build the application, it gives the following error:</p>\n<blockquote>\n<pre class="lang-none prettyprint-override"><code>Cannot implicitly convert type decimal to double\n</code></pre>\n</blockquote>\n<p>I have tried using <code>trans</code> and <code>double</code>, but then the <code>Control</code> doesn\'t work. This code worked fine in a past VB.NET project.</p>\n', 'OwnerUserId': '8', 'LastEditorUserId': '3072350', 'LastEditorDisplayName': 'Rich B', 'LastEditDate': '2021-02-26T03:31:15.027', 'LastActivityDate': '2021-02-26T03:31:15.027', 'Title': 'How to convert a Decimal to a Double in C#?'

In [9]:
def simpleTagCheck(arr):
    return 'is valid' if len(arr) != len(search_for_tags_df.loc[search_for_tags_df['TAG'].
                                                  isin(arr)]) else 'not valid'




In [10]:
search_for_tags_df.loc[search_for_tags_df['TAG'] == 'data-collection']

,TAG
9594,data-collection


In [11]:

#Platforms & tools &libraries
TAGS = ['google-app-maker', 'appian', 'outsystems', 'zoho', 'mendix', 
        'powerapps', 'quickbase', 'vinyl', 'salesforce-lightning', 'github-copilot','Knack','Airtable']

# Research Related
TAGS += ['machine-learning', 'low-code']

# Keywords
TAGS += ['build-automation', 'sdlc', 'automation','integration','classification',
        'workflow','resource-management','user-interface', 'data-collection']



COLS = ["Id", "PostTypeId", "AcceptedAnswerId", "ParentId", "CreationDate",
        "DeletionDate", "Score", "ViewCount", "Body", "LastEditDate", "LastActivityDate",
        "Title", "Tags", "AnswerCount", "CommentCount", "FavoriteCount", "ClosedDate",
        "CommunityOwnedDate", "ContentLicense"]

In [12]:
simpleTagCheck(TAGS)

'is valid'

In [13]:
# Extract data by tags
print("Start Extraction")
postcount =0
with open(OUTPUT1, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(COLS)
    _, root = next(context)
    for index, item in enumerate(context):
        event, elem = item
        # prevent duplicates
        if event == "end" and elem.tag == "row":
            tags = elem.attrib.get('Tags', 'None')
            #remove all '<', '>'
            reworked_tags = tags[1:len(tags)-1].split('><')
            for tag in TAGS:
                if tag in reworked_tags:
                    postcount+=1
                    data = []
                    for col in COLS:
                        data.append(elem.attrib.get(col, ''))
                    csvwriter.writerow(data)
                    continue
            elem.clear()
            root.clear()          
        # log progress every 100000 
        if index % LOGAMOUNT == 0 and index != 0:
            print('prossesed', index, 'total_posts_gahtered', postcount)

        if index > CUTTOFF:
            break
                
            
            
print("Extraction Over")

Start Extraction
prossesed 1000000 total_posts_gahtered 1713
prossesed 2000000 total_posts_gahtered 3168
prossesed 3000000 total_posts_gahtered 4552
prossesed 4000000 total_posts_gahtered 5967
prossesed 5000000 total_posts_gahtered 7275
prossesed 6000000 total_posts_gahtered 8403
prossesed 7000000 total_posts_gahtered 9497
prossesed 8000000 total_posts_gahtered 10562
prossesed 9000000 total_posts_gahtered 11622
prossesed 10000000 total_posts_gahtered 12659


KeyboardInterrupt: 

In [14]:
# clean data
colsSet = set(TAGS)
df = pd.read_csv(OUTPUT1)
df

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense
0,2056,1,101561.0,NaN,2008-08-05T10:06:33.020,NaN,2241,525972,"<p>When looking beyond the <a href=""https://en...",2020-08-29T01:23:49.180,2021-05-19T14:23:15.523,What are MVP and MVC and what is the difference?,<user-interface><model-view-controller><design...,24,5,1324.0,NaN,NaN,CC BY-SA 3.0
1,2222,1,8753.0,NaN,2008-08-05T12:52:42.573,NaN,18,2657,<p>I'm currently working on an application wit...,2014-08-08T13:51:24.583,2014-08-08T13:51:24.583,How can I unit test Flex applications from wit...,<apache-flex><eclipse><unit-testing><build-aut...,4,0,1.0,NaN,NaN,CC BY-SA 3.0
2,2933,1,2937.0,NaN,2008-08-05T22:26:00.797,NaN,289,187921,<p>Python works on multiple platforms and can ...,2020-10-11T06:50:09.827,2021-08-28T08:04:40.667,Create a directly-executable cross-platform GU...,<python><user-interface><deployment><tkinter><...,13,0,213.0,NaN,NaN,CC BY-SA 4.0
3,3362,1,13130.0,NaN,2008-08-06T13:27:05.453,NaN,105,171758,<p>I would like to be able to use the <kbd>Tab...,2019-01-18T08:02:43.230,2019-01-18T08:02:43.230,Capturing TAB key in text box,<javascript><user-interface>,6,2,27.0,2011-02-10T20:27:58.760,NaN,CC BY-SA 4.0
4,3607,1,370131.0,NaN,2008-08-06T16:10:44.313,NaN,16,5236,<p>I've got TotroiseSVN installed and have a m...,2008-08-08T12:01:11.967,2018-12-27T21:50:33.780,Integrating Fogbugz with TortoiseSVN with no U...,<svn><tortoisesvn><integration><fogbugz>,5,2,12.0,NaN,NaN,CC BY-SA 2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12871,6222477,1,NaN,NaN,2011-06-03T02:45:54.340,NaN,0,201,<p>I´m a tutor in a programming introduction c...,2012-07-06T16:28:54.810,2012-07-06T17:34:49.917,Easy way to show a GUI development to a C intr...,<c><user-interface>,2,4,NaN,NaN,NaN,CC BY-SA 3.0
12872,6222590,1,NaN,NaN,2011-06-03T03:12:17.900,NaN,0,428,<h2>Scenario:</h2>\n\n<p>I am developing a web...,2011-06-03T05:56:12.167,2011-06-03T05:56:12.167,Asp.Net Web Based Work Flow,<.net><asp.net><.net-4.0><workflow>,1,2,NaN,NaN,NaN,CC BY-SA 3.0
12873,6223389,1,6463591.0,NaN,2011-06-03T05:40:57.410,NaN,9,8597,<p>Is there a way to do UI changes in a non-UI...,NaN,2014-08-18T16:54:10.723,Do some Android UI stuff in non-UI thread,<java><android><multithreading><user-interface>,6,0,3.0,NaN,NaN,CC BY-SA 3.0
12874,6225503,1,NaN,NaN,2011-06-03T09:49:59.630,NaN,0,950,<p>When I use the code </p>\n\n<pre><code>cvCr...,NaN,2011-06-03T10:18:06.610,cvCreateButton VS2010,<visual-studio-2010><user-interface><opencv>,1,0,NaN,NaN,NaN,CC BY-SA 3.0


In [15]:
print('REMOVE QUESTIONS WITH WRONG TAGS')
print('Before', len(df))
indicesToRemove = []
for i in range(len(df)):
    tags = df.iloc[i]['Tags'][1:-1].replace('><', ' ').split()
    f = True
    for tag in tags:
        if tag in colsSet:
            f = False
            break
    if f:
        indicesToRemove.append(i)
df = df.drop(index=indicesToRemove)

print('After', len(df))

REMOVE QUESTIONS WITH WRONG TAGS
Before 12876
After 12876


In [16]:
print('\nREMOVING DUPLICATE SCORES')
print('Before', len(df))
df.drop_duplicates('Id',inplace=True)
print('After', len(df))




REMOVING DUPLICATE SCORES
Before 12876
After 12712


In [17]:
print('\nREMOVING QUESTIONS WITH NEGATIVE SCORES')
print('Before', len(df))
df = df[df['Score'] >= 0]
df.to_csv(OUTPUT2, index=False)
print('After', len(df))


REMOVING QUESTIONS WITH NEGATIVE SCORES
Before 12712
After 12564


In [18]:
df = pd.read_csv(OUTPUT2)
df

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense
0,2056,1,101561.0,NaN,2008-08-05T10:06:33.020,NaN,2241,525972,"<p>When looking beyond the <a href=""https://en...",2020-08-29T01:23:49.180,2021-05-19T14:23:15.523,What are MVP and MVC and what is the difference?,<user-interface><model-view-controller><design...,24,5,1324.0,NaN,NaN,CC BY-SA 3.0
1,2222,1,8753.0,NaN,2008-08-05T12:52:42.573,NaN,18,2657,<p>I'm currently working on an application wit...,2014-08-08T13:51:24.583,2014-08-08T13:51:24.583,How can I unit test Flex applications from wit...,<apache-flex><eclipse><unit-testing><build-aut...,4,0,1.0,NaN,NaN,CC BY-SA 3.0
2,2933,1,2937.0,NaN,2008-08-05T22:26:00.797,NaN,289,187921,<p>Python works on multiple platforms and can ...,2020-10-11T06:50:09.827,2021-08-28T08:04:40.667,Create a directly-executable cross-platform GU...,<python><user-interface><deployment><tkinter><...,13,0,213.0,NaN,NaN,CC BY-SA 4.0
3,3362,1,13130.0,NaN,2008-08-06T13:27:05.453,NaN,105,171758,<p>I would like to be able to use the <kbd>Tab...,2019-01-18T08:02:43.230,2019-01-18T08:02:43.230,Capturing TAB key in text box,<javascript><user-interface>,6,2,27.0,2011-02-10T20:27:58.760,NaN,CC BY-SA 4.0
4,3607,1,370131.0,NaN,2008-08-06T16:10:44.313,NaN,16,5236,<p>I've got TotroiseSVN installed and have a m...,2008-08-08T12:01:11.967,2018-12-27T21:50:33.780,Integrating Fogbugz with TortoiseSVN with no U...,<svn><tortoisesvn><integration><fogbugz>,5,2,12.0,NaN,NaN,CC BY-SA 2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12559,6222477,1,NaN,NaN,2011-06-03T02:45:54.340,NaN,0,201,<p>I´m a tutor in a programming introduction c...,2012-07-06T16:28:54.810,2012-07-06T17:34:49.917,Easy way to show a GUI development to a C intr...,<c><user-interface>,2,4,NaN,NaN,NaN,CC BY-SA 3.0
12560,6222590,1,NaN,NaN,2011-06-03T03:12:17.900,NaN,0,428,<h2>Scenario:</h2>\n\n<p>I am developing a web...,2011-06-03T05:56:12.167,2011-06-03T05:56:12.167,Asp.Net Web Based Work Flow,<.net><asp.net><.net-4.0><workflow>,1,2,NaN,NaN,NaN,CC BY-SA 3.0
12561,6223389,1,6463591.0,NaN,2011-06-03T05:40:57.410,NaN,9,8597,<p>Is there a way to do UI changes in a non-UI...,NaN,2014-08-18T16:54:10.723,Do some Android UI stuff in non-UI thread,<java><android><multithreading><user-interface>,6,0,3.0,NaN,NaN,CC BY-SA 3.0
12562,6225503,1,NaN,NaN,2011-06-03T09:49:59.630,NaN,0,950,<p>When I use the code </p>\n\n<pre><code>cvCr...,NaN,2011-06-03T10:18:06.610,cvCreateButton VS2010,<visual-studio-2010><user-interface><opencv>,1,0,NaN,NaN,NaN,CC BY-SA 3.0


In [19]:
answerIdDict = {}
for index, row in df.iterrows():
    # access data using column names
    temp = row['AcceptedAnswerId']
    # collect only not nan vlues with indexes
    if not (temp != temp):
        answerIdDict[int(temp)] = index

print('removed nan, reworked total:', len(answerIdDict))



removed nan, reworked total: 8775


In [28]:
context = ET.iterparse(POSTSFILE, events=("start", "end"),
                       parser=ET.XMLParser(encoding='utf-8'))

In [29]:
def updater(header, data, filename):
    with open(filename, "w", newline="", encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)
        writer.writeheader()
        writer.writerows(data)
        
print("Start Processing")
postcount =0 
with open(OUTPUT2, newline='', encoding='utf-8') as file:
    readData = [row for row in csv.DictReader(file)]
    _, root = next(context)
    for index, item in enumerate(context):
        event, elem = item        
        if event == "end" and elem.tag == "row":
            ansId = int(elem.attrib.get('Id', 'None'))
            if ansId in answerIdDict:
                postcount += 1
                
                ansindex = answerIdDict[ansId]
                readData[ansindex]['AcceptedAnswer'] = elem.attrib.get("Body", '')
                readData[ansindex]['AcceptedAnswerCreationDate'] = elem.attrib.get('CreationDate', '')
                readHeader = readData[ansindex].keys()
                updater(readHeader, readData, OUTPUT3)
            elem.clear()
            root.clear()
        if index % LOGAMOUNT == 0 and index != 0:
            print('prossesed:', index, 'total_answers_gahtered:', postcount)
                
        if index > CUTTOFF:
            break
            
print("End Processing")

Start Processing
prossesed: 1000000 total_answers_gahtered: 1131
prossesed: 2000000 total_answers_gahtered: 2113
prossesed: 3000000 total_answers_gahtered: 3057
prossesed: 4000000 total_answers_gahtered: 4037
prossesed: 5000000 total_answers_gahtered: 4904
prossesed: 6000000 total_answers_gahtered: 5651


KeyboardInterrupt: 

In [30]:
df = pd.read_csv(OUTPUT3)
df

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,LastEditDate,...,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense,AcceptedAnswer,AcceptedAnswerCreationDate
0,2056,1,101561.0,NaN,2008-08-05T10:06:33.020,NaN,2241,525972,"<p>When looking beyond the <a href=""https://en...",2020-08-29T01:23:49.180,...,What are MVP and MVC and what is the difference?,<user-interface><model-view-controller><design...,24,5,1324.0,NaN,NaN,CC BY-SA 3.0,<h2>Model-View-Presenter</h2>\n<p>In <strong>M...,2008-09-19T12:46:52.687
1,2222,1,8753.0,NaN,2008-08-05T12:52:42.573,NaN,18,2657,<p>I'm currently working on an application wit...,2014-08-08T13:51:24.583,...,How can I unit test Flex applications from wit...,<apache-flex><eclipse><unit-testing><build-aut...,4,0,1.0,NaN,NaN,CC BY-SA 3.0,"<p>The <a href=""http://code.google.com/p/dpuin...",2008-08-12T12:28:18.340
2,2933,1,2937.0,NaN,2008-08-05T22:26:00.797,NaN,289,187921,<p>Python works on multiple platforms and can ...,2020-10-11T06:50:09.827,...,Create a directly-executable cross-platform GU...,<python><user-interface><deployment><tkinter><...,13,0,213.0,NaN,NaN,CC BY-SA 4.0,<p>First you will need some GUI library with P...,2008-08-05T22:34:25.397
3,3362,1,13130.0,NaN,2008-08-06T13:27:05.453,NaN,105,171758,<p>I would like to be able to use the <kbd>Tab...,2019-01-18T08:02:43.230,...,Capturing TAB key in text box,<javascript><user-interface>,6,2,27.0,2011-02-10T20:27:58.760,NaN,CC BY-SA 4.0,<p>Even if you capture the <code>keydown</code...,2008-08-16T13:55:26.087
4,3607,1,370131.0,NaN,2008-08-06T16:10:44.313,NaN,16,5236,<p>I've got TotroiseSVN installed and have a m...,2008-08-08T12:01:11.967,...,Integrating Fogbugz with TortoiseSVN with no U...,<svn><tortoisesvn><integration><fogbugz>,5,2,12.0,NaN,NaN,CC BY-SA 2.5,<p>I've been investigating this issue and have...,2008-12-16T00:03:30.553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12559,6222477,1,NaN,NaN,2011-06-03T02:45:54.340,NaN,0,201,<p>I´m a tutor in a programming introduction c...,2012-07-06T16:28:54.810,...,Easy way to show a GUI development to a C intr...,<c><user-interface>,2,4,NaN,NaN,NaN,CC BY-SA 3.0,NaN,NaN
12560,6222590,1,NaN,NaN,2011-06-03T03:12:17.900,NaN,0,428,<h2>Scenario:</h2>\n\n<p>I am developing a web...,2011-06-03T05:56:12.167,...,Asp.Net Web Based Work Flow,<.net><asp.net><.net-4.0><workflow>,1,2,NaN,NaN,NaN,CC BY-SA 3.0,NaN,NaN
12561,6223389,1,6463591.0,NaN,2011-06-03T05:40:57.410,NaN,9,8597,<p>Is there a way to do UI changes in a non-UI...,NaN,...,Do some Android UI stuff in non-UI thread,<java><android><multithreading><user-interface>,6,0,3.0,NaN,NaN,CC BY-SA 3.0,NaN,NaN
12562,6225503,1,NaN,NaN,2011-06-03T09:49:59.630,NaN,0,950,<p>When I use the code </p>\n\n<pre><code>cvCr...,NaN,...,cvCreateButton VS2010,<visual-studio-2010><user-interface><opencv>,1,0,NaN,NaN,NaN,CC BY-SA 3.0,NaN,NaN


In [31]:
questionIdDict = {}
for index, row in df.iterrows():
    temp = row['Id']
    if not (temp != temp):
        questionIdDict[int(temp)] = index
        
print('removed nan, reworked total:',len(questionIdDict))

removed nan, reworked total: 12564


In [32]:
questionIdDict

{2056: 0,
 2222: 1,
 2933: 2,
 3362: 3,
 3607: 4,
 3725: 5,
 3781: 6,
 3867: 7,
 4850: 8,
 4913: 9,
 5242: 10,
 5313: 11,
 5329: 12,
 5855: 13,
 6085: 14,
 6729: 15,
 7174: 16,
 7440: 17,
 7489: 18,
 8004: 19,
 8214: 20,
 8968: 21,
 9433: 22,
 9846: 23,
 10059: 24,
 10634: 25,
 10901: 26,
 11152: 27,
 11263: 28,
 11462: 29,
 12009: 30,
 12428: 31,
 12523: 32,
 12815: 33,
 13347: 34,
 13607: 35,
 14893: 36,
 15053: 37,
 15056: 38,
 15815: 39,
 16145: 40,
 16199: 41,
 17880: 42,
 17922: 43,
 18197: 44,
 18719: 45,
 18869: 46,
 19113: 47,
 19185: 48,
 19314: 49,
 19387: 50,
 19883: 51,
 22326: 52,
 22798: 53,
 24196: 54,
 24580: 55,
 24813: 56,
 24823: 57,
 24941: 58,
 26706: 59,
 27674: 60,
 28826: 61,
 29423: 62,
 29555: 63,
 29855: 64,
 30101: 65,
 30318: 66,
 30521: 67,
 32103: 68,
 32966: 69,
 33079: 70,
 33321: 71,
 33349: 72,
 33703: 73,
 33909: 74,
 34109: 75,
 34286: 76,
 34304: 77,
 34325: 78,
 34711: 79,
 35762: 80,
 35907: 81,
 35922: 82,
 36108: 83,
 36568: 84,
 36646: 85,
 3

In [33]:
context = ET.iterparse(POSTSFILE, events=("start", "end"),
                       parser=ET.XMLParser(encoding='utf-8'))

In [34]:
print("Start Processing")
postcount =0 
with open(OUTPUT4, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(COLS)
    _, root = next(context)
    for index, item in enumerate(context):
        event, elem = item
        if event == "end" and elem.tag == "row":
            parentId = int(elem.attrib.get('ParentId', '0'))
            
            if parentId in questionIdDict:
                postcount += 1
                data = []
                for col in COLS:
                    data.append(elem.attrib.get(col, ''))
                csvwriter.writerow(data)
                continue
            
        
            elem.clear()
            root.clear()
        # progress
        if index % LOGAMOUNT == 0 and index != 0:
            print('prossesed:', index, 'total_questions_gahtered:', postcount)
                
        if index > CUTTOFF:
            break
print("End Processing")
 

Start Processing
prossesed: 1000000 total_questions_gahtered: 6335
prossesed: 2000000 total_questions_gahtered: 10783


KeyboardInterrupt: 

In [35]:
df = pd.read_csv(OUTPUT3, low_memory=False)

answers_df = df.copy()
answers_df = answers_df[['Id', 'AcceptedAnswer']]
answers_df = answers_df.dropna()
answers_df.columns = ['id', 'raw']
answers_df.insert(1, 'qa', 'a')
answers_df.id = answers_df.id.apply(np.int64)
print(answers_df.head())
print(len(answers_df))

questions_df = df.copy()
questions_df = questions_df[['Id', 'Body', 'Title']]
questions_df['titlePlusQuestion'] = questions_df[['Title', 'Body']].apply(lambda x: ' '.join(x), axis=1)
questions_df = questions_df.drop(['Title', 'Body'], axis=1)
questions_df.columns = ['id', 'raw']
questions_df.insert(1, 'qa', 'q')
print(questions_df.head())
print(len(questions_df))

df = pd.concat([questions_df,answers_df])
print(len(df))
df.to_csv(PROCESSEDFINAL, index=False)

     id qa                                                raw
0  2056  a  <h2>Model-View-Presenter</h2>\n<p>In <strong>M...
1  2222  a  <p>The <a href="http://code.google.com/p/dpuin...
2  2933  a  <p>First you will need some GUI library with P...
3  3362  a  <p>Even if you capture the <code>keydown</code...
4  3607  a  <p>I've been investigating this issue and have...
6083
     id qa                                                raw
0  2056  q  What are MVP and MVC and what is the differenc...
1  2222  q  How can I unit test Flex applications from wit...
2  2933  q  Create a directly-executable cross-platform GU...
3  3362  q  Capturing TAB key in text box <p>I would like ...
4  3607  q  Integrating Fogbugz with TortoiseSVN with no U...
12564
18647
